In [1]:
import pandas as pd
import sys
import os
from os import listdir
os.chdir("c:/Users/Robert/Documents/Projekte/dev/sport_betting/")
import config as CONFIG
pd.set_option('display.max_columns', 500)

from src.features import load_team_opponent
from src.features.preprocess import derived_odds,get_bookmaker, get_odd_pred_team_opponent, split_date,add_last_3_scores_column
import numpy as np

# Lade Daten ein

## Team Opponent

In [2]:
from src.utils.utils import load_from_pickle
df = load_from_pickle(source_path= CONFIG.DATA_FOLDER_RAW+'main_leagues_database_raw.sav')
df = df[~df.FTR.isna()]

In [64]:
bet_attributes = ['IWH','IWD','IWA','BWH','BWD','BWA','B365H','B365D','B365A']
df['Date'] = pd.to_datetime(df['Date'])
complete_bets =  df[bet_attributes].isna().T.sum()==0
df = df[complete_bets] 

X = df[["HomeTeam","AwayTeam","Div","B365H","B365D","B365A","BWH","BWD","BWA","IWH","IWD","IWA"]]
X.head()

y = df[["B365H","B365D","B365A","FTR"]]

y.columns =["H","D","A","R"]
#y = y.reset_index()



In [71]:
import pandas as pd

def make_y(y_row):
    r = y_row["R"]
    y_row[r] = y_row[r]-1
    results = {'H','D','A'}
    y_row[list(results - {r})] = 0
    return y_row

In [73]:
y.isna().sum()

H    0
D    0
A    0
R    0
dtype: int64

In [74]:
# apply the make_y function to every row of the DataFrame
y= y.apply(lambda x: make_y(x), axis=1)

y = y.drop("R",axis=1)
# print the updated DataFrame
print(y)


       H    D     A
0    0.0  0.0  0.61
1    0.0  2.5  0.00
2    0.5  0.0  0.00
3    0.0  0.0  1.20
4    0.0  0.0  1.40
..   ...  ...   ...
235  0.0  2.6  0.00
236  1.2  0.0  0.00
237  0.0  2.6  0.00
238  0.0  0.0  1.55
239  0.0  0.0  3.33

[71218 rows x 3 columns]


In [75]:
import numpy as np

def set_max_row(arr):
    """
    Sets the maximum value in each row of a numpy array to 1 and all other values to 0.
    """
    # Find indices of maximum values in each row
    max_indices = np.argmax(arr, axis=1)

    # Set all values to 0 except for the maximum value in each row
    for i in range(arr.shape[0]):
        max_index = max_indices[i]
        arr[i, :] = 0
        arr[i, max_index] = 1

    return arr



NameError: name 'y_test' is not defined

In [82]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from pytorch_tabnet.tab_model import TabNetRegressor
import torch

# Load dataset

# Convert categorical variables to numerical labels
le = LabelEncoder()
for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = le.fit_transform(X[col])

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=42)

# Define TabNet model
model = TabNetRegressor(n_d=32, n_a=32, n_steps=5, gamma=1.3, lambda_sparse=0, optimizer_fn=torch.optim.Adam)

# Train model
model.fit(X_train.values, y_train.values, max_epochs=5, patience=10, batch_size=1024)

# Make predictions
preds = model.predict(X_test.values)


C:\Users\Robert\AppData\Roaming\Python\Python37\site-packages\pytorch_tabnet\abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
C:\Users\Robert\AppData\Roaming\Python\Python37\site-packages\pytorch_tabnet\abstract_model.py:651: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 2.02339 |  0:00:10s
epoch 1  | loss: 1.30232 |  0:00:22s
epoch 2  | loss: 1.28201 |  0:00:32s
epoch 3  | loss: 1.27658 |  0:00:42s
epoch 4  | loss: 1.27505 |  0:00:51s


In [87]:
import torch
from pytorch_tabnet.metrics import Metric

class Precision(Metric):
    def __init__(self):
        self._name = "precision"
        self._maximize = True

    def __call__(self, y_true, y_pred):
        score = max(y_true) - max(y_pred)
        return score

In [90]:
preds

array([[0.5122517 , 0.72616965, 0.6397993 ],
       [0.48821676, 0.7725003 , 0.626613  ],
       [0.67565805, 0.80637556, 0.45570093],
       ...,
       [0.36489636, 0.55527186, 0.39439783],
       [0.332025  , 0.7137349 , 0.65184796],
       [0.46311593, 0.6343225 , 0.61849713]], dtype=float32)

In [91]:
Precision(preds, y_test.values)

TypeError: __init__() takes 1 positional argument but 3 were given

In [ ]:
wins = y_test.sum(axis=1)
output = (set_max_row(y_test.values)*set_max_row(preds)).sum(axis= 1)*wins
print(wins)

In [ ]:
pd.Series(model.feature_importances_,index=X.columns)

In [ ]:
import pandas as pd
import numpy as np

def set_max_row(df):
    """
    Sets everything but the maximum value in each row of a pandas dataframe to 0.
    """
    # Convert dataframe to numpy array
    arr = df.values

    # Find indices of maximum values in each row
    max_indices = np.argmax(arr, axis=1)

    # Set all values before the maximum to 0
    for i in range(arr.shape[0]):
        max_index = max_indices[i]
        arr[i, :max_index] = 0

    # Set all values after the maximum to 0
    for i in range(arr.shape[0]):
        max_index = max_indices[i]
        arr[i, max_index+1:] = 0

    # Convert back to pandas dataframe
    result_df = pd.DataFrame(arr, columns=df.columns, index=df.index)

    return result_df


In [ ]:
sum(df.B365_Opponent.isna())

## Sortiere den Datensatz nach Datum, für die Durchschnittsfelder

In [ ]:
df = split_date(df)
df = df.sort_values(['year','month','day'],ascending=True)
df = df.reset_index()
df.Date = pd.to_datetime(df.Date)

In [ ]:
df["diff_FTG_against_Opponent"] = df["FTG_Team"]-df["FTG_Opponent"]
df["diff_ShotsOnTarget_against_Opponent"] = df["ShotsOnTarget_Team"]-df["ShotsOnTarget_Opponent"]

# Erstelle Zielvariable

In [ ]:
df['Target'] = df.FTG_Team>df.FTG_Opponent
df['Target'].value_counts()

# Features entwicklen

In [ ]:
standad_attributes = ['Target','atHome','Date','Div']
bookmaker_attributes = get_bookmaker(bm = 'BW',df = df)+get_bookmaker(bm='B365',df = df)+get_bookmaker(bm='IW',df = df)
bet_on_team = get_bookmaker(bm='Team',df=df[bookmaker_attributes])
bet_on_opponent = get_bookmaker(bm='Opponent',df=df[bookmaker_attributes])
bet_on_draw = get_bookmaker(bm='Draw',df=df[bookmaker_attributes])


In [ ]:
df_derived_b365 = get_odd_pred(bet="B365",df=df)
df_derived_iw = get_odd_pred(bet="IW",df=df)
df_derived_bw = get_odd_pred(bet="BW",df=df)


In [ ]:
df_derived_team = derived_odds(sight = 'Team',df=df, odds=bet_on_team)
df_derived_opponent = derived_odds(sight = 'Opponent',df=df, odds=bet_on_opponent)
df_derived_draw = derived_odds(sight = 'Draw',df=df, odds=bet_on_draw)


In [ ]:
# df = add_last_3_scores_column(df = df, score_column="diff_FTG_against_Opponent",anz_games=4)
# df = add_last_3_scores_column(df = df, score_column="diff_ShotsOnTarget_against_Opponent",anz_games=4)
# df = add_last_3_scores_column(df = df, score_column="B365_Team",anz_games=4)
# df = add_last_3_scores_column(df = df, score_column="B365_Opponent",anz_games=4)
# df = df.dropna()
# rolling_mean_attributes = [c for c in  df.columns if 'last' in c]

# Modelldatensatz erstellen

In [ ]:
model_data = pd.concat([df[standad_attributes],
                        #df[rolling_mean_attributes],
                        df[bookmaker_attributes],
                        df[["Team","Opponent","month","year"]],
                        df_derived_team,
                        df_derived_opponent,
                        df_derived_draw,
                        df_derived_b365,
                        df_derived_iw,
                        df_derived_bw],axis=1)

In [ ]:
model_data.replace([np.inf, -np.inf], np.nan, inplace=True)

#model_data = model_data.dropna()

In [ ]:
df.IW_Team.isna().sum()

## Encoding Teams

In [ ]:
from sklearn import preprocessing
le_div = preprocessing.LabelEncoder()
le_team = preprocessing.LabelEncoder()


In [ ]:
le_team.fit(list(set(model_data.Team)))
model_data.Team = le_team.transform(model_data.Team)
model_data.Opponent = le_team.transform(model_data.Opponent)

In [ ]:
le_div.fit(list(set(model_data.Div)))
model_data.Div = le_div.transform(model_data.Div)


Aufteilen in Training, Test und Validierung

In [ ]:
model_data.Date = pd.to_datetime(model_data.Date)

In [ ]:
import datetime
datum_1_1_2020 = datetime.datetime(2021, 6, 1)
test_data = model_data[model_data.Date>datum_1_1_2020]
model_data = model_data[model_data.Date<datum_1_1_2020]


In [ ]:
datum_1_1_2020 = datetime.datetime(2019, 6, 1)
model_data_valid = model_data[model_data.Date>datum_1_1_2020]
model_data_train = model_data[model_data.Date<datum_1_1_2020]

In [ ]:
len(model_data_train)

In [ ]:
model_data_train.to_csv(CONFIG.DATA_FOLDER_PROCESSED + 'Train/model_data.csv')
model_data_valid.to_csv(CONFIG.DATA_FOLDER_PROCESSED + 'Valid/model_data.csv')
test_data.to_csv(CONFIG.DATA_FOLDER_PROCESSED + 'Test/model_data.csv')
